In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import tensorflow as tf
import json, gensim, sklearn, pickle, sys, re, os
import IPython.display as ipd

In [4]:
from gensim.parsing.preprocessing import preprocess_documents
from gensim.parsing.preprocessing import preprocess_string
from gensim.models.phrases import Phrases
from gensim.parsing.preprocessing import strip_tags, strip_short, strip_multiple_whitespaces, stem_text
from gensim.parsing.preprocessing import remove_stopwords
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
from gensim import utils
from scholarly import scholarly
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from IPython.display import Audio
sound_file = './Music/invalid_keypress.mp3'

In [5]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

# First we load the 5 kinds of document vectors #

In [4]:
glove_file = datapath('/home/ubuntu/abstracts_roberta_epoch1_second_v3.txt')
tmp_file = get_tmpfile("/home/ubuntu/abstracts_roberta_epoch1_second_v3_gensim.txt")
_ = glove2word2vec(glove_file, tmp_file)
sentence_model_second = KeyedVectors.load_word2vec_format(tmp_file,)

In [5]:
glove_file = datapath('/home/ubuntu/abstracts_roberta_epoch1_first_v3.txt')
tmp_file = get_tmpfile("/home/ubuntu/abstracts_roberta_epoch1_first_v3_gensim.txt")
_ = glove2word2vec(glove_file, tmp_file)
sentence_model_first = KeyedVectors.load_word2vec_format(tmp_file,)

In [6]:
glove_file = datapath('/home/ubuntu/abstracts_roberta_epoch1_last4_concat_v3.txt')
tmp_file = get_tmpfile("/home/ubuntu/abstracts_roberta_epoch1_last_concat_v3_gensim.txt")
_ = glove2word2vec(glove_file, tmp_file)
sentence_model_concat = KeyedVectors.load_word2vec_format(tmp_file,)

In [7]:
glove_file = datapath('/home/ubuntu/abstracts_roberta_epoch1_embeddings_v3.txt')
tmp_file = get_tmpfile("/home/ubuntu/abstracts_roberta_epoch1_embeddings_v3_gensim.txt")
_ = glove2word2vec(glove_file, tmp_file)
sentence_model_zero = KeyedVectors.load_word2vec_format(tmp_file,)

In [8]:
glove_file = datapath('/home/ubuntu/abstracts_roberta_epoch1_last4_average_v3.txt')
tmp_file = get_tmpfile("/home/ubuntu/abstracts_roberta_epoch1_last_average_v3_gensim.txt")
_ = glove2word2vec(glove_file, tmp_file)
sentence_model_average = KeyedVectors.load_word2vec_format(tmp_file,)

In [4]:
glove_file = datapath('/home/ubuntu/abstracts_roberta_epoch1_last4_last')
tmp_file = get_tmpfile("/home/ubuntu/abstracts_roberta_epoch1_last4_last_gensim.txt")
_ = glove2word2vec(glove_file, tmp_file)
_ = glove2word2vec(glove_file, tmp_file)
sentence_model_last = KeyedVectors.load_word2vec_format(tmp_file,)

In [7]:
#glove_file = datapath('/home/ubuntu/abstracts_roberta_epoch1_last4_concat')
#tmp_file = get_tmpfile("/home/ubuntu/abstracts_roberta_epoch1_last4_concat_gensim.txt")
#_ = glove2word2vec(glove_file, tmp_file)
#sentence_model_e2 = KeyedVectors.load_word2vec_format(tmp_file,)

In [8]:
#glove_file = datapath('/home/ubuntu/abstracts_roberta_epoch1_last4_average')
#tmp_file = get_tmpfile("/home/ubuntu/abstracts_roberta_epoch1_last4_average_gensim.txt")
#_ = glove2word2vec(glove_file, tmp_file)
#sentence_model_average = KeyedVectors.load_word2vec_format(tmp_file,)

In [6]:
#glove_file = datapath('/home/ubuntu/abstracts_roberta_epoch1_embeddings')
#tmp_file = get_tmpfile("/home/ubuntu/abstracts_roberta_epoch1_embeddings_gensim.txt")
#_ = glove2word2vec(glove_file, tmp_file)
#sentence_model_embed = KeyedVectors.load_word2vec_format(tmp_file,)

TF-IDF preparation

In [5]:
datapd=pd.read_json("arxiv-metadata-oai-snapshot.json",  lines=True)
datapd_hep_th = datapd.loc[datapd['categories'].str.contains('hep-th')]
hep_abstracts = datapd_hep_th[ list(datapd_hep_th.iloc[:,0:1]) + ['abstract'] + ['title'] + ['authors']]

In [ ]:
hep_corpus=hep_abstracts['abstract'].values
processed_hep=preprocess_documents(hep_corpus)
hep_dictionary = gensim.corpora.Dictionary(processed_hep)
hep_bow_corpus = [hep_dictionary.doc2bow(text) for text in processed_hep]
hep_tfidf = gensim.models.TfidfModel(hep_bow, smartirs='npu')
index = gensim.similarities.MatrixSimilarity(hep_tfidf[hep_bow])

In [11]:
def recommendgpt(model,string,num):
    for i in range(num):
        x=model.wv.similar_by_word(string,topn=num)[i][0]
        print(f"{i+1}) Paper title : {hep_abstracts.loc[hep_abstracts['id'] == x].title.values}. \n Abstract : {hep_abstracts.loc[hep_abstracts['id'] == x].abstract.values}. \n Authors : {hep_abstracts.loc[hep_abstracts['id'] == x].authors.values}. \n Arxiv ID : {hep_abstracts.loc[hep_abstracts['id'] == x].id.values} \n \n ")
        i=i+1

In [20]:
recommendgpt(sentence_model_zero,"1212.3616",30)

/home/ubuntu/anaconda3/envs/aws_neuron_tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


1) Paper title : ['Bounds on OPE Coefficients from Interference Effects in the Conformal\n  Collider']. 
 Abstract : ['  We apply the average null energy condition to obtain upper bounds on the\nthree-point function coefficients of stress tensors and a scalar operator,\n$\\langle TT {\\cal O } \\rangle,$ in general CFTs. We also constrain the\ngravitational anomaly of $U(1)$ currents in four-dimensional CFTs, which are\nencoded in three-point functions of the form $\\langle TT J \\rangle$. In\ntheories with a large $N$ AdS dual we translate these bounds into constraints\non the coefficient of a higher derivative bulk term of the form $\\int\n\\phi\\hspace{.5mm} W^2 $. We speculate that these bounds also apply in\nde-Sitter. In this case our results constrain inflationary observables, such as\nthe amplitude for chiral gravity waves that originate from higher derivative\nterms in the Lagrangian of the form $\\phi \\hspace{.5mm}W W^*$.\n']. 
 Authors : ['Clay Cordova, Juan Maldacena, Gust

11) Paper title : ['Moving the CFT into the bulk with $T\\bar T$']. 
 Abstract : ['  Recent work by Zamolodchikov and others has uncovered a solvable irrelevant\ndeformation of general 2D CFTs, defined by turning on the dimension 4 operator\n$T \\bar T$, the product of the left- and right-moving stress tensor. We propose\nthat in the holographic dual, this deformation represents a geometric cutoff\nthat removes the asymptotic region of AdS and places the QFT on a Dirichlet\nwall at finite radial distance $r = r_c$ in the bulk. As a quantitative check\nof the proposed duality, we compute the signal propagation speed, energy\nspectrum, and thermodynamic relations on both sides. In all cases, we obtain a\nprecise match. We derive an exact RG flow equation for the metric dependence of\nthe effective action of the $T \\bar T$ deformed theory, and find that it\ncoincides with the Hamilton-Jacobi equation that governs the radial evolution\nof the classical gravity action in AdS.\n']. 
 Author

19) Paper title : ['Cornering gapless quantum states via their torus entanglement']. 
 Abstract : ["  The entanglement entropy (EE) has emerged as an important window into the\nstructure of complex quantum states of matter. We analyze the universal part of\nthe EE for gapless systems put on tori in 2d/3d, denoted by $\\chi$. Focusing on\nscale invariant systems, we derive general non-perturbative properties for the\nshape dependence of $\\chi$, and reveal surprising relations to the EE\nassociated with corners in the entangling surface. We obtain closed-form\nexpressions for $\\chi$ in 2d/3d within a model that arises in the study of\nconformal field theories (CFTs), and use them to obtain ansatzes without\nfitting parameters for the 2d/3d free boson CFTs. Our numerical lattice\ncalculations show that the ansatzes are highly accurate. Finally, we discuss\nhow the torus EE can act as a fingerprint of exotic states such as gapless\nquantum spin liquids, e.g. Kitaev's honeycomb model.\n"]

27) Paper title : ['Quantum Strings and the AdS4/CFT3 Interpolating Function']. 
 Abstract : ['  The existence of a nontrivial interpolating function h(\\lambda) is one of the\nnovel features of the new AdS4/CFT3 correspondence involving ABJM theory. At\nstrong coupling, most of the investigation of semiclassical effects so far has\nbeen for strings in the AdS4 sector. Several cutoff prescriptions have been\nproposed, leading to different predictions for the constant term in the\nexpansion h(\\lambda)=\\sqrt{\\lambda/2} + c + ... . We calculate quantum\ncorrections for giant magnons, using the algebraic curve, and show by comparing\nto the dispersion relation that the same prescriptions lead to the same values\nof c in this CP3 sector. We then turn to finite-J effects, where a comparison\nwith the Luescher F-term correction shows a mismatch for one of the three sum\nprescriptions. We also compute some dyonic and higher F-terms for future\ncomparisons.\n']. 
 Authors : ['Michael C. Abbo

In [13]:
recommendgpt(sentence_model_zero,"1908.10996",30)

/home/ubuntu/anaconda3/envs/aws_neuron_tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


1) Paper title : ['Islands outside the horizon']. 
 Abstract : ['  We consider an AdS$_2$ black hole in equilibrium with a bath, which we take\nto have a dual description as (0+1)-dimensional quantum mechanical system\ncoupled to a (1+1)-dimensional field theory serving as the bath. We compute the\nentropies of both the quantum mechanical degrees of freedom and of the bath\nseparately, while allowing contributions from entanglement wedge "islands". We\nfind situations where the island extends {\\it outside} the black hole horizon.\nThis suggests possible causality paradoxes which we show are avoided because of\nthe quantum focusing conjecture. Finally, we formulate a version of the\ninformation paradox for a black hole in contact with a bath in the\nHartle-Hawking state, and demonstrate the role of islands in resolving this\nparadox.\n']. 
 Authors : ['Ahmed Almheiri, Raghu Mahajan, Juan Maldacena']. 
 Arxiv ID : ['1910.11077'] 
 
 
2) Paper title : ['Very General Holographic Supercond

11) Paper title : ['Black hole entropy from Quantum Geometry']. 
 Abstract : ['  Quantum Geometry (the modern Loop Quantum Gravity using graphs and\nspin-networks instead of the loops) provides microscopic degrees of freedom\nthat account for the black-hole entropy. However, the procedure for state\ncounting used in the literature contains an error and the number of the\nrelevant horizon states is underestimated. In our paper a correct method of\ncounting is presented. Our results lead to a revision of the literature of the\nsubject. It turns out that the contribution of spins greater then 1/2 to the\nentropy is not negligible. Hence, the value of the Barbero-Immirzi parameter\ninvolved in the spectra of all the geometric and physical operators in this\ntheory is different than previously derived. Also, the conjectured relation\nbetween Quantum Geometry and the black hole quasi-normal modes should be\nunderstood again.\n']. 
 Authors : ['Marcin Domagala and Jerzy Lewandowski (Uniwersyt

19) Paper title : ['Delocalizing Entanglement of Anisotropic Black Branes']. 
 Abstract : ['  We study the mutual information between pairs of regions on the two\nasymptotic boundaries of maximally-extended anisotropic black-brane solutions.\nThis quantity characterizes the local pattern of entanglement of thermofield\ndouble states which are dual to these geometries. We analyse the disruption of\nthe mutual information in anisotropic shock wave geometries and show that the\nentanglement velocity plays an important role in this phenomenon. Besides that\nwe compute several chaos-related properties of this system, like the\nentanglement velocity, the butterfly velocity and the scrambling time. We find\nthat the butterfly velocity and the entanglement velocity violate the upper\nbounds proposed in 1311.1200 and 1612.00082, but remain bounded by their\ncorresponding values in the infrared effective theory.\n']. 
 Authors : ['Viktor Jahnke']. 
 Arxiv ID : ['1708.07243'] 
 
 
20) Paper title

29) Paper title : ['Entanglement entropy for free scalar fields in AdS']. 
 Abstract : ['  We compute entanglement entropy for free massive scalar fields in anti-de\nSitter (AdS) space. The entangling surface is a minimal surface whose boundary\nis a sphere at the boundary of AdS. The entropy can be evaluated from the\nthermal free energy of the fields on a topological black hole by using the\nreplica method. In odd-dimensional AdS, exact expressions of the Renyi entropy\n$S_n$ are obtained for arbitrary $n$. We also evaluate 1-loop corrections\ncoming from the scalar fields to holographic entanglement entropy. Applying the\nresults, we compute the leading difference of entanglement entropy between two\nholographic CFTs related by a renormalization group flow triggered by a double\ntrace deformation. The difference is proportional to the shift of a central\ncharge under the flow.\n']. 
 Authors : ['Sotaro Sugishita']. 
 Arxiv ID : ['1608.00305'] 
 
 
30) Paper title : ['Weak cosmic cen

In [12]:
# recommendgpt(sentence_model_last,"1908.10996",30) old 

/home/ubuntu/anaconda3/envs/aws_neuron_tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


1) Paper title : ['Angle deficit & non-local gravitoelectromagnetism around a slowly\n  spinning cosmic string']. 
 Abstract : ["  Cosmic strings, as remnants of the symmetry breaking phase in the Early\nUniverse, may be susceptible to non-local physics. Here we show that the\npresence of a Poincar\\'e-invariant non-locality---parametrized by a factor\n$\\exp(-\\Box\\ell^2)$---regularizes the gravitational field and thereby changes\nthe properties of spacetime: it is now simply connected and the angle deficit\naround the cosmic string becomes a function of the radial distance. Similar\nchanges occur for the non-local gravitomagnetic field of a rotating cosmic\nstring, and we translate these mathematical facts into the language of\nnon-local gravitoelectromagnetism and thereby provide a physical\ninterpretation. We hope that these insights might prove helpful in the search\nfor traces of non-local physics in our Universe.\n"]. 
 Authors : ['Jens Boos']. 
 Arxiv ID : ['2003.13847'] 
 
 


9) Paper title : ['Generation of Cosmological Seed Magnetic Fields from Inflation with\n  Cutoff']. 
 Abstract : ['  Inflation has the potential to seed the galactic magnetic fields observed\ntoday. However, there is an obstacle to the amplification of the quantum\nfluctuations of the electromagnetic field during inflation: namely the\nconformal invariance of electromagnetic theory on a conformally flat underlying\ngeometry. As the existence of a preferred minimal length breaks the conformal\ninvariance of the background geometry, it is plausible that this effect could\ngenerate some electromagnetic field amplification. We show that this scenario\nis equivalent to endowing the photon with a large negative mass during\ninflation. This effective mass is negligibly small in a radiation and matter\ndominated universe. Depending on the value of the free parameter of the theory,\nwe show that the seed required by the dynamo mechanism can be generated. We\nalso show that this mechanism can pr

17) Paper title : ['A phase-integral perspective on alpha-decay']. 
 Abstract : ['  This paper applies the phase-integral method to the stationary theory of\nalpha-decay. The rigorous form of the connection formulae, and their\none-directional nature that was not widely known in the physical literature,\nare applied. The condition for obtaining s-wave metastable states affects the\nstationary state at large distance from the nucleus, which is dominated by the\ncosine of the phase integral minus (pi over 4). Accurate predictions for the\nlowest s-wave metastable state and mean life of the radioactive nucleus are\nobtained in the case of Uranium. The final part of the paper describes the\nphase-integral algorithm for evaluating stationary states by means of a\nsuitable choice of freely specifiable base function. Within this framework, an\noriginal approximate formula for the phase integrand with arbitrary values of\nthe angular momentum quantum number is obtained.\n']. 
 Authors : ['Giam

25) Paper title : ['Inflation with a Planck-scale frequency cutoff']. 
 Abstract : ['  The implementation of a Planck-scale high frequency and short wavelength\ncutoff in quantum theories on expanding backgrounds may have potentially\nnontrivial implications, such as the breaking of local Lorentz invariance and\nthe existence of a yet unknown mechanism for the creation of vacuum modes. In\nscenarios where inflation begins close to the cutoff scale, these effects could\nhave observable consequences as trans-Planckian modes are redshifted to\ncosmological scales. In close analogy with similar studies of Hawking\nradiation, a simple theory of a minimally coupled scalar field in de Sitter\nspace is studied, with a high frequency cutoff imposed by a nonlinear\ndispersion relation. Under certain conditions the model predicts deviations\nfrom the standard inflationary scenario. We also comment on the difficulties in\ngeneralizing fluid models of Hawking radiation to cosmological space-times.\

In [13]:
recommendgpt(sentence_model_e2,"1908.10996",30)

/home/ubuntu/anaconda3/envs/aws_neuron_tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


1) Paper title : ['Anisotropic Unruh temperatures']. 
 Abstract : ['  The relative entropy between very high energy localized excitations and the\nvacuum, where both states are reduced to a spatial region, gives place to a\nprecise definition of a local temperature produced by vacuum entanglement\nacross the boundary. This generalizes the Unruh temperature of the Rindler\nwedge to arbitrary regions. The local temperatures can be read off from the\nshort distance leading terms in the modular Hamiltonian. For free scalar and\nfermion fields they have a universal geometric expression that follows by\nsolving a particular eikonal type equation in Euclidean space. This equation\ngeneralizes to any dimension the holomorphic property that holds in two\ndimensions. For regions of arbitrary shapes the local temperatures at a point\nare direction dependent. We compute their explicit expression for the geometry\nof a wall or strip.\n']. 
 Authors : ['Raul Arias, Horacio Casini, Marina Huerta, Die

10) Paper title : ['LQ Stars : Modifying the Tolman -- Oppenheimer -- Volkoff equations a la\n  LQC']. 
 Abstract : ["  We rewrite the Tolman -- Oppenheimer -- Volkoff (TOV) equations for four and\nhigher dimensional static spherically symmetric stars so that they resemble the\nequations for anisotropic cosmology. This becomes possible by treating the\ncurvature terms of the sphere as part of the matter sector. Comparing then with\nthe effective equations in the Loop Quantum Cosmology -- inspired models, we\npropose analogous modifications to the TOV equations which contain one\narbitrary function. A linear function gives the TOV equations. If the function\nand all its derivatives are finite then the solutions are non singular. For\n$sin \\; x \\;$, an `image star' appears beyond the central region of maximal\npressures in the original star. This is evocative of a bouncing universe in\nLoop Quantum Cosmology. We discuss possible observational consequences of these\nfeatures.\n"]. 
 Aut

19) Paper title : ['Exact self-duality in a modified Skyrme model']. 
 Abstract : ['  We propose a modification of the Skyrme model that supports a self-dual\nsector possessing exact non-trivial finite energy solutions. The action of such\na theory possesses the usual quadratic and quartic terms in field derivatives,\nbut the couplings of the components of the Maurer-Cartan form of the Skyrme\nmodel is made by a non-constant symmetric matrix, instead of the usual Killing\nform of the SU(2) Lie algebra. The introduction of such a matrix make the\nself-duality equations conformally invariant in three space dimensions, even\nthough it may break the global internal symmetries of the original Skyrme\nmodel. For the case where that matrix is proportional to the identity we show\nthat the theory possesses exact self-dual Skyrmions of unity topological\ncharges.\n']. 
 Authors : ['L. A. Ferreira']. 
 Arxiv ID : ['1705.01824'] 
 
 
20) Paper title : ['Energy loss at zero temperature from extrem

29) Paper title : ['Calculation of multi-loop superstring amplitudes']. 
 Abstract : ['  Multi-loop interaction amplitudes in the theory of the closed, oriented\nsuperstrings are obtained by the integration of local amplitudes which are\nrepresented by a sum of the spinning string local amplitudes. The last local\namplitudes are given explicitly through super-Schottky group parameters and\ninteraction vertex coordinates on the $(1|1)$ complex supermanifold. The\nintegration is ambiguous under those replacements of the integration variables\nwhich admix Grassmann variables to the boson ones. So the calculation is guided\nby a preservation of local symmetries of the superstring. The obtained\namplitudes are free from divergences and consistent with the world-sheet\nsymmetries. The vacuum amplitude and 1-, 2- and 3-point amplitudes of massless\nstates vanish once the integration over certin modular variables and\ninteraction vertex coordinates.\n']. 
 Authors : ['G. S. Danilov']. 
 Arxiv 

In [14]:
recommendgpt(sentence_model_average,"1908.10996",30)

/home/ubuntu/anaconda3/envs/aws_neuron_tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


1) Paper title : ['Anisotropic Unruh temperatures']. 
 Abstract : ['  The relative entropy between very high energy localized excitations and the\nvacuum, where both states are reduced to a spatial region, gives place to a\nprecise definition of a local temperature produced by vacuum entanglement\nacross the boundary. This generalizes the Unruh temperature of the Rindler\nwedge to arbitrary regions. The local temperatures can be read off from the\nshort distance leading terms in the modular Hamiltonian. For free scalar and\nfermion fields they have a universal geometric expression that follows by\nsolving a particular eikonal type equation in Euclidean space. This equation\ngeneralizes to any dimension the holomorphic property that holds in two\ndimensions. For regions of arbitrary shapes the local temperatures at a point\nare direction dependent. We compute their explicit expression for the geometry\nof a wall or strip.\n']. 
 Authors : ['Raul Arias, Horacio Casini, Marina Huerta, Die

9) Paper title : ['Black Holes, Entanglement and Random Matrices']. 
 Abstract : ['  We provide evidence that strong quantum entanglement between Hilbert spaces\ndoes not generically create semiclassical wormholes between the corresponding\ngeometric regions in the context of the AdS/CFT correspondence. We propose a\ndescription of low-energy gravity probes as random operators on the space of\nblack hole states. We use this description to compute correlators between the\nentangled systems, and argue that a wormhole can only exist if correlations are\nlarge. Conversely, we also argue that large correlations can exist in the\nmanifest absence of a Lorentzian wormhole. Thus the strength of the\nentanglement cannot generically diagnose spacetime connectedness, without\ninformation on the spectral properties of the probing operators. Our random\nmatrix picture of probes also provides suggestive insights into the problem of\n"seeing behind a horizon".\n']. 
 Authors : ['Vijay Balasubramanian

17) Paper title : ['Energy loss at zero temperature from extremal black holes']. 
 Abstract : ['  Using the AdS/CFT correspondence, we probe the extremal black holes by\nstudying the energy loss of a moving heavy point particle in a strongly-coupled\nboundary field theory at zero temperature and finite charge density. We first\nconsider the extremal Reissner-Nordstr$\\rm\\ddot{o}$m-AdS (AdSRN) background\nblack hole in the bulk and find an analytic solution of drag force which\ndepends on the finite charge density of the boundary field theory. By studying\nthe near horizon geometry of trailing string in the bulk, we show the IR\ndivergency does not show the logarithmic behavior. We study also the stochastic\nbehavior of the probe and find the Langevin diffusion coefficients. This study\nis extended to the extremal backgrounds with vanishing entropy density in\nappropriate extremal limits.\n']. 
 Authors : ['Moslem Ahmadvand, Kazem Bitaghsir Fadafan']. 
 Arxiv ID : ['1512.05290'] 
 
 
1

25) Paper title : ['Non-perturbative Quantum Field Theory and the Geometry of Functional\n  Spaces']. 
 Abstract : ['  In this paper we construct a non-commutative geometry over a configuration\nspace of gauge connections and show that it gives rise to an interacting,\nnon-perturbative quantum gauge theory coupled to a fermionic field on a curved\nbackground. The non-commutative geometry is given by an infinite-dimensional\nBott-Dirac type operator, whose square gives the Hamilton operator, and which\ninteracts with an algebra generated by holonomy-diffeomorphisms. The Bott-Dirac\noperator and the associated Hilbert space relies on a metric on the\nconfiguration space of connections, which effectively works as a covariant\nultra-violet regulator. We show that the construction coincides with\nperturbative quantum field theory in a local limit. Questions concerning\nLorentz invariance and the fermionic sector are left open.\n']. 
 Authors : ['Johannes Aastrup and Jesper M. Grimstrup']. 


In [15]:
recommendgpt(sentence_model_embed,"1908.10996",30)

/home/ubuntu/anaconda3/envs/aws_neuron_tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


1) Paper title : ['Islands outside the horizon']. 
 Abstract : ['  We consider an AdS$_2$ black hole in equilibrium with a bath, which we take\nto have a dual description as (0+1)-dimensional quantum mechanical system\ncoupled to a (1+1)-dimensional field theory serving as the bath. We compute the\nentropies of both the quantum mechanical degrees of freedom and of the bath\nseparately, while allowing contributions from entanglement wedge "islands". We\nfind situations where the island extends {\\it outside} the black hole horizon.\nThis suggests possible causality paradoxes which we show are avoided because of\nthe quantum focusing conjecture. Finally, we formulate a version of the\ninformation paradox for a black hole in contact with a bath in the\nHartle-Hawking state, and demonstrate the role of islands in resolving this\nparadox.\n']. 
 Authors : ['Ahmed Almheiri, Raghu Mahajan, Juan Maldacena']. 
 Arxiv ID : ['1910.11077'] 
 
 
2) Paper title : ['Very General Holographic Supercond

11) Paper title : ['Conditions on holographic entangling surfaces in higher curvature\n  gravity']. 
 Abstract : ['  We study the extremal surfaces of functionals recently proposed for the\nholographic calculation of entanglement entropy in general higher curvature\ntheories, using New Massive gravity and Gauss-Bonnet gravity as concrete\nexamples. We show that the entropy functionals admit closed extremal surfaces,\nwhich for black hole backgrounds can encircle the event horizon of the black\nhole. In the examples considered, such closed surfaces correspond to a lower\nvalue of the entropy functional than expected from CFT calculations, implying a\nseeming mismatch between the bulk and boundary calculations. For Lorentzian\nsettings we show that this problem can be resolved by imposing a causality\nconstraint on the extremal surfaces. The possibility of deriving conditions\nfrom an alternative conical boundary condition method as proposed by Lewkowycz\nand Maldacena is explored.\n']. 

19) Paper title : ['On a new relation between entanglement and geometry from M(atrix) theory']. 
 Abstract : ["  In the context of Matrix/light-cone gauge M-theory, we develop a new approach\nfor computing quantum entanglement between a probe gravitating in the vicinity\nof a source mass and the source mass. We demonstrate that this entanglement is\nrelated to the gravitational potential energy between the two objects. We then\nshow that the Von Neumann entropy is a function of two derivatives of the\ngravitational potential. We conjecture a relation between the entropy and the\nlocal Riemann tensor sampled by the probe, establishing a general scheme to\nrelate entropy to local geometric data. This relation connects the rate of\nchange, rotation, and twist of a small volume element at the probe's location\nto the quantum entanglement of the probe with the source.\n"]. 
 Authors : ['Vatche Sahakian']. 
 Arxiv ID : ['1902.04229'] 
 
 
20) Paper title : ['Thermodynamics in Black-hole/CFT 

29) Paper title : ['Holographic Cosmology and Uncertainty Relation']. 
 Abstract : ['  It is believed that a primary principle of the theory of quantum gravity is\nthe Holographic Principle according to which a physical system can be described\nonly by degrees of freedom living on its boundary. The generalized covariant\nformulation of the principle considers the entropy content on truncated\nlight-sheets: light-like hypersurfaces of non-positive expansion orthogonally\ngenerated from a boundary. When we construct the truncated light-sheets for\ncosmological observers we find a general expression for the minimum cosmic time\ninterval from the apparent horizon to verify the holographic prescription; this\nminimum time is related to quantum effects involved in the entropy computation.\nFinally, we arrive to the uncertainty relation from the Holographic Principle,\nwhich suggests a deep connection between general covariance, entropy bounds and\nquantum mechanics.\n']. 
 Authors : ['M. A. 

In [16]:
recommendgpt(sentence_model_embed,"1212.3616",30)

/home/ubuntu/anaconda3/envs/aws_neuron_tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


1) Paper title : ['Bounds on OPE Coefficients from Interference Effects in the Conformal\n  Collider']. 
 Abstract : ['  We apply the average null energy condition to obtain upper bounds on the\nthree-point function coefficients of stress tensors and a scalar operator,\n$\\langle TT {\\cal O } \\rangle,$ in general CFTs. We also constrain the\ngravitational anomaly of $U(1)$ currents in four-dimensional CFTs, which are\nencoded in three-point functions of the form $\\langle TT J \\rangle$. In\ntheories with a large $N$ AdS dual we translate these bounds into constraints\non the coefficient of a higher derivative bulk term of the form $\\int\n\\phi\\hspace{.5mm} W^2 $. We speculate that these bounds also apply in\nde-Sitter. In this case our results constrain inflationary observables, such as\nthe amplitude for chiral gravity waves that originate from higher derivative\nterms in the Lagrangian of the form $\\phi \\hspace{.5mm}W W^*$.\n']. 
 Authors : ['Clay Cordova, Juan Maldacena, Gust

11) Paper title : ['Moving the CFT into the bulk with $T\\bar T$']. 
 Abstract : ['  Recent work by Zamolodchikov and others has uncovered a solvable irrelevant\ndeformation of general 2D CFTs, defined by turning on the dimension 4 operator\n$T \\bar T$, the product of the left- and right-moving stress tensor. We propose\nthat in the holographic dual, this deformation represents a geometric cutoff\nthat removes the asymptotic region of AdS and places the QFT on a Dirichlet\nwall at finite radial distance $r = r_c$ in the bulk. As a quantitative check\nof the proposed duality, we compute the signal propagation speed, energy\nspectrum, and thermodynamic relations on both sides. In all cases, we obtain a\nprecise match. We derive an exact RG flow equation for the metric dependence of\nthe effective action of the $T \\bar T$ deformed theory, and find that it\ncoincides with the Hamilton-Jacobi equation that governs the radial evolution\nof the classical gravity action in AdS.\n']. 
 Author

19) Paper title : ['Interacting fields in real-time AdS/CFT']. 
 Abstract : ['  We compute time-ordered 2- and 3-pt correlation functions of CFT scalar\noperators between generic in/out states. The calculation is holographically\ncarried out by considering a non backreacting AdS scalar field with a $\\lambda\n\\phi^3$ self-interaction term on a combination of Euclidean and Lorentzian AdS\nsections following the Skenderis-van Rees prescription. We show that, although\nworking in an essentially different set up, the final result for the 3-pt\ncorrelators agree with those of Rastelli et al. for Euclidean AdS. By analyzing\nthe inner product between the in/out excited states in the large $N$\napproximation, we argue that a cubic bulk interaction deforms the excited\nstates from coherent into \\emph{squeezed}. Finally, a diagrammatic\ninterpretation of the results suggests some general properties for the\n$n$-point correlation functions between excited states.\n']. 
 Authors : ["Marcelo Bot

27) Paper title : ['G-inflation: inflation driven by the Galileon field']. 
 Abstract : ['  We propose a new class of inflation model, G-inflation, which has a\nGalileon-like nonlinear derivative interaction of the form $G(\\phi,\n(\\nabla\\phi)^2)\\Box\\phi$ in the Lagrangian with the resultant equations of\nmotion being of second order. It is shown that (almost) scale-invariant\ncurvature fluctuations can be generated even in the exactly de Sitter\nbackground and that the tensor-to-scalar ratio can take a significantly larger\nvalue than in the standard inflation models, violating the standard consistency\nrelation. Furthermore, violation of the null energy condition can occur without\nany instabilities. As a result, the spectral index of tensor modes can be blue,\nwhich makes it easier to observe quantum gravitational waves from inflation by\nthe planned gravitational-wave experiments such as LISA and DECIGO as well as\nby the upcoming CMB experiments such as Planck and CMBpol.\n'].

In [17]:
recommendgpt(sentence_model_e2,"1212.3616",30)

/home/ubuntu/anaconda3/envs/aws_neuron_tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


1) Paper title : ['Semiclassical string computation of strong-coupling corrections to\n  dimensions of operators in Konishi multiplet']. 
 Abstract : ['  Following our earlier work in arXiv:0906.4294 we show how to use\nsemiclassical string quantization approach to compute the leading corrections\nto the energy of AdS_5 x S^5 string states on the first excited string level\nthat should correspond to operators in the Konishi multiplet of N=4 SYM theory.\nCompared to examples in our previous paper the string solutions we consider\nhere carry an extra component of S^5 angular momentum J. This facilitates their\nidentification with operators in the Konishi multiplet. We show that for such\nstring states with J=2 the coefficient of the subleading lambda^(-1/4) term in\nthe large string tension expansion of the energy is twice the one found in\narXiv:0906.4294. The resulting value matches the one found for the Konishi\nstate in the sl(2) sector from the Y-system/TBA approach, resolving an ap

10) Paper title : ['QCD-like Theories at Finite Baryon and Isospin Density']. 
 Abstract : ['  We use 2-color QCD as a model to study the effects of simultaneous presence\nof chemical potentials for isospin charge, $\\mu_I$, and for baryon number,\n$\\mu_B$. We determine the phase diagrams for 2 and 4 flavor theories using the\nmethod of effective chiral Lagrangians at low densities and weak coupling\nperturbation theory at high densities. We determine the values of various\ncondensates and densities as well as the spectrum of excitations as functions\nof $\\mu_I$ and $\\mu_B$. A similar analysis of QCD with quarks in the adjoint\nrepresentation is also presented. Our results can be of relevance for lattice\nsimulations of these theories. We predict a phase of inhomogeneous condensation\n(Fulde-Ferrel-Larkin-Ovchinnikov phase) in the 2 colour 2 flavor theory, while\nwe do not expect it the 4 flavor case or in other realizations of QCD with a\npositive measure.\n']. 
 Authors : ['K. Spl

18) Paper title : ['Deformations of Cosmological Solutions of D=11 Supergravity']. 
 Abstract : ['  We study Lunin-Maldacena deformations of cosmological backgrounds of D=11\nsupergravity which gives an easy way to generate solutions with nonzero 4-form\nflux starting from solutions of pure Einstein equations which possess at least\nthree U(1) isometries. We illustrate this on the vacuum S-brane solution from\nwhich the usual SM2-brane solution is obtained. Applying the method again, one\neither gets the recently found S-brane system where contribution of the\nChern-Simons term to field equations is non-zero or the SM2-SM2(0)\nintersection, depending on which U(1) directions are used during the process.\nRepeated usage of the procedure gives rise to configurations with several\noverlapping S-branes some of which are new. We also employ this method to\nconstruct two more new solutions and make comments about accelerating\ncosmologies that follow from such deformed solutions after compac

26) Paper title : ['On thermal Nieh-Yan anomaly in topological Weyl materials']. 
 Abstract : ['  We discuss the possibility of a gravitional Nieh-Yan anomaly of the type\n$\\partial_\\mu j^\\mu_5 =\\gamma T^2{\\cal T}^a\\wedge {\\cal T}_a$ in topological\nWeyl materials, where $T$ is temperature and ${\\cal T}^a$ is the effective or\nemergent torsion. As distinct from the non-universal parameter $\\Lambda$ in the\nconventional (zero temperature) Nieh-Yan anomaly -- with canonical dimensions\nof momentum -- the parameter $\\gamma$ is dimensionless. This suggests that the\ndimensionless parameter is fundamental, being determined by the geometry,\ntopology and number of the number of chiral quantum fields without any explicit\nnon-universal UV scales. This conforms with previous results in the literature,\nas well as spectral flow calculations using torsional magnetic field at finite\ntemperature.\n']. 
 Authors : ['J. Nissinen and G.E. Volovik']. 
 Arxiv ID : ['1911.03382'] 
 
 
27) Pap

In [18]:
recommendgpt(sentence_model_average,"1212.3616",30)

/home/ubuntu/anaconda3/envs/aws_neuron_tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


1) Paper title : ['Semiclassical string computation of strong-coupling corrections to\n  dimensions of operators in Konishi multiplet']. 
 Abstract : ['  Following our earlier work in arXiv:0906.4294 we show how to use\nsemiclassical string quantization approach to compute the leading corrections\nto the energy of AdS_5 x S^5 string states on the first excited string level\nthat should correspond to operators in the Konishi multiplet of N=4 SYM theory.\nCompared to examples in our previous paper the string solutions we consider\nhere carry an extra component of S^5 angular momentum J. This facilitates their\nidentification with operators in the Konishi multiplet. We show that for such\nstring states with J=2 the coefficient of the subleading lambda^(-1/4) term in\nthe large string tension expansion of the energy is twice the one found in\narXiv:0906.4294. The resulting value matches the one found for the Konishi\nstate in the sl(2) sector from the Y-system/TBA approach, resolving an ap

9) Paper title : ['Chiral phase structure of three flavor QCD at vanishing baryon number\n  density']. 
 Abstract : ['  We investigate the phase structure of QCD with 3 degenerate quark flavors as\nfunction of the degenerate quark masses at vanishing baryon number density. We\nuse the Highly Improved Staggered Quarks on lattices with temporal extent\n$N_{t}=6$ and perform calculations for six values of quark masses, which in the\ncontinuum limit correspond to pion masses in the range $80~{\\rm MeV} \\lesssim\nm_{\\pi} \\lesssim 230~$MeV. By analyzing the volume and temperature dependence\nof the chiral condensate and chiral susceptibility we find no direct evidence\nfor a first order phase transition in this range of pion mass values. Relying\non the universal scaling behaviors of the chiral observables near an\nanticipated chiral critical point, we estimate an upper bound for the critical\npion mass, $m_\\pi^c \\lesssim$ 50 MeV, below which a region of first order\nchiral phase transi

17) Paper title : ['One-Loop BPS amplitudes as BPS-state sums']. 
 Abstract : ["  Recently, we introduced a new procedure for computing a class of one-loop\nBPS-saturated amplitudes in String Theory, which expresses them as a sum of\none-loop contributions of all perturbative BPS states in a manifestly T-duality\ninvariant fashion. In this paper, we extend this procedure to all BPS-saturated\namplitudes of the form \\int_F \\Gamma_{d+k,d} {\\Phi}, with {\\Phi} being a weak\n(almost) holomorphic modular form of weight -k/2. We use the fact that any such\n{\\Phi} can be expressed as a linear combination of certain absolutely\nconvergent Poincar\\'e series, against which the fundamental domain F can be\nunfolded. The resulting BPS-state sum neatly exhibits the singularities of the\namplitude at points of gauge symmetry enhancement, in a chamber-independent\nfashion. We illustrate our method with concrete examples of interest in\nheterotic string compactifications.\n"]. 
 Authors : ['Carlo

25) Paper title : ['Non-global Logarithms at 3 Loops, 4 Loops, 5 Loops and Beyond']. 
 Abstract : ['  We calculate the coefficients of the leading non-global logarithms for the\nhemisphere mass distribution analytically at 3, 4, and 5 loops at large Nc . We\nconfirm that the integrand derived with the strong-energy-ordering\napproximation and fixed-order iteration of the Banfi-Marchesini-Syme (BMS)\nequation agree. Our calculation exploits a hidden PSL(2,R) symmetry associated\nwith the jet directions, apparent in the BMS equation after a stereographic\nprojection to the Poincare disk. The required integrals have an iterated form,\nleading to functions of uniform transcendentality. This allows us to extract\nthe coefficients, and some functional dependence on the jet directions, by\ncomputing the symbols and coproducts of appropriate expressions involving\nclassical and Goncharov polylogarithms. Convergence of the series to a\nnumerical solution of the BMS equation is also discussed.\n

In [163]:
print(index_small)

MatrixSimilarity<1000 docs, 3858 features>


In [164]:
import time 

In [ ]:
pickle.dump(index_list,open("/home/ubuntu/index_list_Mar10","wb"))

In [66]:
starter=sentence_model_zero.similar_by_word("1212.3616",topn=1000)

In [38]:
tij=hep_abstracts
tij = tij.reset_index(drop=True)
def fastsimplerecommend(string,string1,cutoff):
    pointer3=hep_abstracts.loc[(hep_abstracts['id'].str.match(string))]
    queryvalue3=pointer3['abstract'].values
    processed_query3=preprocess_documents(queryvalue3)
    test_bow_corpus3 = [hep_dictionary.doc2bow(text) for text in processed_query3]
    vec_test_bow3=hep_tfidf[test_bow_corpus3]
    sims = index[vec_test_bow3[0]]
    ind=tij[tij['id'] == string1].index[0]
    if sims[ind]>cutoff:
        return (1,sims[ind])
    return (0,0) 

def fast_roberta_recommender(model,orig,num,cutoff):
    i=1
    j=0
    starter=model.similar_by_word(orig,topn=1000)
    while i<num:
        x=starter[j][0]
        if fastsimplerecommend(orig,x,cutoff)[0]==1:
            print(f"\n {i}) With TFIDF score {fastsimplerecommend(orig,x,cutoff)[1]} and Roberta rank {j} ")
            print(f" Paper title : {hep_abstracts.loc[hep_abstracts['id'] == x].title.values}. \n Abstract : {hep_abstracts.loc[hep_abstracts['id'] == x].abstract.values}. \n Authors : {hep_abstracts.loc[hep_abstracts['id'] == x].authors.values}. \n Arxiv ID : {hep_abstracts.loc[hep_abstracts['id'] == x].id.values} \n \n ")
            i=i+1
        j=j+1

In [33]:
fast_gpt_tfidf_recommender(sentence_model_zero,"1908.10996")


 1) With TFIDF score 0.2728791832923889 and GPT-2 rank 0 
 Paper title : ['Islands outside the horizon']. 
 Abstract : ['  We consider an AdS$_2$ black hole in equilibrium with a bath, which we take\nto have a dual description as (0+1)-dimensional quantum mechanical system\ncoupled to a (1+1)-dimensional field theory serving as the bath. We compute the\nentropies of both the quantum mechanical degrees of freedom and of the bath\nseparately, while allowing contributions from entanglement wedge "islands". We\nfind situations where the island extends {\\it outside} the black hole horizon.\nThis suggests possible causality paradoxes which we show are avoided because of\nthe quantum focusing conjecture. Finally, we formulate a version of the\ninformation paradox for a black hole in contact with a bath in the\nHartle-Hawking state, and demonstrate the role of islands in resolving this\nparadox.\n']. 
 Authors : ['Ahmed Almheiri, Raghu Mahajan, Juan Maldacena']. 
 Arxiv ID : ['1910.11077'] 


KeyboardInterrupt: 

In [ ]:
fast_gpt_tfidf_recommender(sentence_model_first,"1908.10996")

In [31]:
fast_gpt_tfidf_recommender(sentence_model_average,"1908.10996")


 1) With TFIDF score 0.11754905432462692 and GPT-2 rank 3 
 Paper title : ['Entropic force in black hole binaries and its Newtonian limits']. 
 Abstract : ["  We give an exact solution for the static force between two black holes at the\nturning points in their binary motion. The results are derived by Gibbs'\nprinciple and the Bekenstein-Hawking entropy applied to the apparent horizon\nsurfaces in time-symmetric initial data. New power laws are derived for the\nentropy jump in mergers, while Newton's law is shown to derive from a new\nadiabatic variational principle for the Hilbert action in the presence of\napparent horizon surfaces. In this approach, entropy is strictly monotonic such\nthat gravity is attractive for all separations including mergers, and the\nBekenstein entropy bound is satisfied also at arbitrarily large separations,\nwhere gravity reduces to Newton's law. The latter is generalized to point\nparticles in the Newtonian limit by application of Gibbs' principle to\nw


 9) With TFIDF score 0.18028882145881653 and GPT-2 rank 45 
 Paper title : ['Time Evolution of Entanglement Entropy in Holographic FLRW Cosmologies']. 
 Abstract : ['  To understand the time-dependent quantum correlation in expanding universes,\nwe study the time-dependent entanglement entropy in the braneworld model. If we\ntake into account a generalized string cloud geometry caused by uniformly\ndistributed open strings, cosmologies on the braneworld result in the standard\nFriedmann-Lema\\^{i}tre-Robertson-Walker cosmologies with various matter\ncontents. On the dual field theory side, open strings are reinterpreted as a\nfundamental matter, while the black hole mass corresponds to the excitation\nenergy of massless gauge bosons. In this work, we show how the string cloud\ngeometry is matched to various braneworld cosmologie,s for example, eternal\ninflation, radiation-, and matter-dominated universes. Then, we investigate how\nthe entanglement entropy evolves in those expanding u


 17) With TFIDF score 0.1550658941268921 and GPT-2 rank 83 
 Paper title : ['Brane Cosmology and the self-tuning of the cosmological constant in the\n  presence of bulk black holes']. 
 Abstract : ['  Motivated by the holographic self-tuning proposal of the cosmological\nconstant, we generalize and study the cosmology of brane-worlds embedded in a\nhigher-dimensional bulk black hole geometry. We describe the equations and\nmatching conditions in the case of flat, spherical and hyperbolic slicing of\nthe bulk geometry and find the conditions for the existence of a static\nsolution. We solve the equations that govern dynamical geometries in the probe\nbrane limit and we describe in detail the resulting brane-world cosmologies. Of\nparticular interest are the properties of solutions when the brane-world\napproaches the black hole horizon. In this case the geometry induced on the\nbrane is that of de Sitter, whose entropy and temperature is related to those\nof the higher dimensional bulk


 25) With TFIDF score 0.1479305624961853 and GPT-2 rank 154 
 Paper title : ['Thermodynamics of Dilatonic Black Holes in $n$ Dimensions']. 
 Abstract : ["  We present a formalism for studying the thermodynamics of black holes in\ndilaton gravity. The thermodynamic variables are defined on a quasilocal\nsurface surrounding the black hole system and are obtained from a general class\nof Lagrangians involving a dilaton. The formalism thus accommodates a large\nnumber of possible theories and black hole spacetimes. Many of the\nthermodynamic quantities are identified from the contribution of the action on\nthe quasilocal boundary. The entropy is found using path integral techniques,\nand a first law of thermodynamics is obtained. As an illustration, we calculate\nthe thermodynamic quantities for two black hole solutions in $(1+1)$\ndimensions: one obtained from a string inspired theory and the other being a\nLiouville black hole in the ``$R=\\kappa T$'' theory with a Liouville field.\n"].

In [26]:
fast_gpt_tfidf_recommender(sentence_model_average,"1212.3616")


 1) With TFIDF score 0.11292888224124908 and GPT-2 rank 12 
 Paper title : ['Regge meets collinear in strongly-coupled $\\mathcal{N} = 4$ super\n  Yang-Mills']. 
 Abstract : ['  We revisit the calculation of the six-gluon remainder function in planar\n$\\mathcal{N} = 4$ super Yang-Mills theory from the strong coupling TBA in the\nmulti-Regge limit and identify an infinite set of kinematically subleading\nterms. These new terms can be compared to the strong coupling limit of the\nfinite-coupling expressions for the impact factor and the BFKL eigenvalue\nproposed by Basso et al. in arXiv:1407.3766, which were obtained from an\nanalytic continuation of the Wilson loop OPE. After comparing the results order\nby order in those subleading terms, we show that it is possible to precisely\nmap both formalisms onto each other. A similar calculation can be carried out\nfor the seven-gluon amplitude, the result of which shows that the central\nemission vertex does not become trivial at strong cou


 9) With TFIDF score 0.10546843707561493 and GPT-2 rank 83 
 Paper title : ['Strong Coupling Expansion for the Conformal Pomeron/Odderon Trajectories']. 
 Abstract : ['  From the perspective of AdS/CFT the Pomeron is identified with a Reggeized\nGraviton, while the Odderons correspond to Reggeized anti-symmetric $AdS_5$\nKalb-Ramond tensor-fields. In this paper, we consider the strong coupling\nexpansion of the dimension of the leading twist operators dual to these Regge\ntrajectories, $\\Delta(j)$, to determine its analytic continuation in $j$ beyond\nthe diffusion limit. In particular, we compute the strong coupling expansion of\nthe intercept to order $\\lambda^{-3}$, where $\\lambda$ is the t\'Hooft coupling,\nfor both the Pomeron, which is $C=+1$ crossing-even, and the "Odderons", which\nare the leading $C=-1$ crossing-odd Regge singularities. We discuss the\nspectral curves of the class of single-trace operators to which these string\nmodes couple.\n']. 
 Authors : ["Richard C. 


 17) With TFIDF score 0.2129826843738556 and GPT-2 rank 251 
 Paper title : ['Nonlinear superhorizon curvature perturbation in generic single-field\n  inflation']. 
 Abstract : ['  We develop a theory of nonlinear cosmological perturbations on superhorizon\nscales for generic single-field inflation. Our inflaton is described by the\nLagrangian of the form $W(X,\\phi)-G(X,\\phi)\\Box\\phi$ with\n$X=-\\partial^{\\mu}\\phi\\partial_{\\mu}\\phi/2$, which is no longer equivalent to a\nperfect fluid. This model is more general than k-inflation, and is called\nG-inflation. A general nonlinear solution for the metric and the scalar field\nis obtained at second order in gradient expansion. We derive a simple master\nequation governing the large-scale evolution of the nonlinear curvature\nperturbation. It turns out that the nonlinear evolution equation is deduced as\na straightforward extension of the corresponding linear equation for the\ncurvature perturbation on uniform $\\phi$ hypersurfaces


 25) With TFIDF score 0.10445020347833633 and GPT-2 rank 371 
 Paper title : ['Toward a 3d Ising model with a weakly-coupled string theory dual']. 
 Abstract : ['  It has long been expected that the 3d Ising model can be thought of as a\nstring theory, where one interprets the domain walls that separate up spins\nfrom down spins as two-dimensional string worldsheets. The usual Ising\nHamiltonian measures the area of these domain walls. This theory has string\ncoupling of unit magnitude. We add new local terms to the Ising Hamiltonian\nthat further weight each spin configuration by a factor depending on the genus\nof the corresponding domain wall, resulting in a new 3d Ising model that has a\ntunable bare string coupling $g_s$. We use a combination of analytical and\nnumerical methods to analyze the phase structure of this model as $g_s$ is\nvaried. We study statistical properties of the topology of worldsheets and\ndiscuss the prospects of using this new deformation at weak string cou

In [28]:
fast_gpt_tfidf_recommender(sentence_model_concat,"1212.3616")


 1) With TFIDF score 0.11292888224124908 and GPT-2 rank 10 
 Paper title : ['Regge meets collinear in strongly-coupled $\\mathcal{N} = 4$ super\n  Yang-Mills']. 
 Abstract : ['  We revisit the calculation of the six-gluon remainder function in planar\n$\\mathcal{N} = 4$ super Yang-Mills theory from the strong coupling TBA in the\nmulti-Regge limit and identify an infinite set of kinematically subleading\nterms. These new terms can be compared to the strong coupling limit of the\nfinite-coupling expressions for the impact factor and the BFKL eigenvalue\nproposed by Basso et al. in arXiv:1407.3766, which were obtained from an\nanalytic continuation of the Wilson loop OPE. After comparing the results order\nby order in those subleading terms, we show that it is possible to precisely\nmap both formalisms onto each other. A similar calculation can be carried out\nfor the seven-gluon amplitude, the result of which shows that the central\nemission vertex does not become trivial at strong cou


 9) With TFIDF score 0.15435536205768585 and GPT-2 rank 108 
 Paper title : ['Scalar fields and three-point functions in D=3 higher spin gravity']. 
 Abstract : ["  We compute boundary three-point functions involving two scalars and a gauge\nfield of arbitrary spin in the AdS vacuum of Vasiliev's higher spin gravity,\nfor any deformation parameter \\lambda. In the process, we develop tools for\nextracting scalar field equations in arbitrary higher spin backgrounds. We work\nin the context of hs[\\lambda] + hs[\\lambda] Chern-Simons theory coupled to\nscalar fields, and make efficient use of the associative lone-star product\nunderlying the hs[\\lambda] algebra. Our results for the correlators precisely\nmatch expectations from CFT; in particular they match those of any CFT with\nW_{\\infinity}[\\lambda] symmetry at large central charge, and with primary\noperators dual to the scalar fields. As this is expected to include the `t\nHooft limit of the W_N minimal model CFT, our results se


 17) With TFIDF score 0.14339226484298706 and GPT-2 rank 270 
 Paper title : ['Singularities of N=1 Supersymmetric Gauge Theory and Matrix Models']. 
 Abstract : ['  In N=1 supersymmetric U(N) gauge theory with adjoint matter $\\Phi$ and\npolynomial tree-level superpotential $W(\\Phi)$, the massless fluctuations about\neach quantum vacuum are generically described by $U(1)^n$ gauge theory for some\nn. However, by tuning the parameters of $W(\\Phi)$ to non-generic values, we can\nreach singular vacua where additional fields become massless. Using both the\nmatrix model prescription and the strong-coupling approach, we study in detail\nthree examples of such singularities: the singularities of the n=1 branch,\nintersections of n=1 and n=2 branches, and a class of N=1 Argyres-Douglas\npoints. In all three examples, we find that the matrix model description of the\nlow-energy physics breaks down in some way at the singularity.\n']. 
 Authors : ['David Shih']. 
 Arxiv ID : ['hep-th/0308001

KeyboardInterrupt: 

In [29]:
fast_gpt_tfidf_recommender(sentence_model_zero,"1212.3616")


 1) With TFIDF score 0.16614513099193573 and GPT-2 rank 0 
 Paper title : ['Bounds on OPE Coefficients from Interference Effects in the Conformal\n  Collider']. 
 Abstract : ['  We apply the average null energy condition to obtain upper bounds on the\nthree-point function coefficients of stress tensors and a scalar operator,\n$\\langle TT {\\cal O } \\rangle,$ in general CFTs. We also constrain the\ngravitational anomaly of $U(1)$ currents in four-dimensional CFTs, which are\nencoded in three-point functions of the form $\\langle TT J \\rangle$. In\ntheories with a large $N$ AdS dual we translate these bounds into constraints\non the coefficient of a higher derivative bulk term of the form $\\int\n\\phi\\hspace{.5mm} W^2 $. We speculate that these bounds also apply in\nde-Sitter. In this case our results constrain inflationary observables, such as\nthe amplitude for chiral gravity waves that originate from higher derivative\nterms in the Lagrangian of the form $\\phi \\hspace{.5mm}W W


 9) With TFIDF score 0.3079841136932373 and GPT-2 rank 28 
 Paper title : ['Scalar-Fermion Analytic Bootstrap in 4D']. 
 Abstract : ['  In this work we discuss an analytic bootstrap approach [1,2] in the context\nof spinning 4D conformal blocks [3,4]. As an example we study the simplest\nspinning case, the scalar-fermion correlator\n$\\langle\\phi\\psi\\phi\\bar\\psi\\rangle$. We find that to every pair of primary\nscalar $\\phi$ and fermion $\\psi$ correspond two infinite towers of fermionic\nlarge spin primary operators. We compute their twists and products of OPE\ncoefficients using both s-t and u-t bootstrap equations to the leading and\nsub-leading orders. We find that the leading order is represented by the\nscalar-fermion generalized free theory and the sub-leading order is governed by\nthe minimal twist bosonic (light scalars, currents and the energy-momentum\ntensor) and fermionic (light fermions and the suppersymmetric current)\noperators present in the spectrum.\n']. 
 Auth


 17) With TFIDF score 0.12770013511180878 and GPT-2 rank 60 
 Paper title : ['An AdS_3 Dual for Minimal Model CFTs']. 
 Abstract : ["  We propose a duality between the 2d W_N minimal models in the large N 't\nHooft limit, and a family of higher spin theories on AdS_3. The 2d CFTs can be\ndescribed as WZW coset models, and include, for N=2, the usual Virasoro unitary\nseries. The dual bulk theory contains, in addition to the massless higher spin\nfields, two complex scalars (of equal mass). The mass is directly related to\nthe 't Hooft coupling constant of the dual CFT. We give convincing evidence\nthat the spectra of the two theories match precisely for all values of the 't\nHooft coupling. We also show that the RG flows in the 2d CFT agree exactly with\nthe usual AdS/CFT prediction of the gravity theory. Our proposal is in many\nways analogous to the Klebanov-Polyakov conjecture for an AdS_4 dual for the\nsinglet sector of large N vector models.\n"]. 
 Authors : ['Matthias R Gaberdie


 25) With TFIDF score 0.12736444175243378 and GPT-2 rank 107 
 Paper title : ['Lessons from crossing symmetry at large N']. 
 Abstract : ['  We consider the four-point correlator of the stress tensor multiplet in N=4\nSYM. We construct all solutions consistent with crossing symmetry in the limit\nof large central charge c ~ N^2 and large g^2 N. While we find an infinite\ntower of solutions, we argue most of them are suppressed by an extra scale\n\\Delta_{gap} and are consistent with the upper bounds for the scaling dimension\nof unprotected operators observed in the numerical superconformal bootstrap at\nlarge central charge. These solutions organize as a double expansion in 1/c and\n1/\\Delta_{gap}. Our solutions are valid to leading order in 1/c and to all\norders in 1/\\Delta_{gap} and reproduce, in particular, instanton corrections\npreviously found. Furthermore, we find a connection between such upper bounds\nand positivity constraints arising from causality in flat space. Finall

In [30]:
fast_gpt_tfidf_recommender(sentence_model_first,"1212.3616")


 1) With TFIDF score 0.13549210131168365 and GPT-2 rank 0 
 Paper title : ['Inflection Point Quintessence']. 
 Abstract : ['  We examine models in which the accelerated expansion of the universe is\ndriven by a scalar field rolling near an inflection point in the potential. For\nthe simplest such models, in which the potential is of the form V(\\phi) = V_0 +\nV_3 (\\phi-\\phi_0)^3, the scalar field can either evolve toward \\phi = \\phi_0 at\nlate times, yielding an asymptotic de Sitter expansion, or it can transition\nthrough the inflection point, producing a transient period of acceleration. We\ndetermine the parameter ranges which produce each of these two possibilities\nand also map out the region in parameter space for which the equation of state\nof the scalar field is close to -1 at all times up to the present, mimicking\n\\LambdaCDM. We show that the latter can be consistent with either eternal or\ntransient acceleration. More complicated inflection point models are also\ninve


 9) With TFIDF score 0.28560900688171387 and GPT-2 rank 33 
 Paper title : ['Resummation at finite conformal spin']. 
 Abstract : ['  We generalize the computation of anomalous dimension and correction to OPE\ncoefficients at finite conformal spin considered recently in\n\\cite{arXiv:1806.10919, arXiv:1808.00612} to arbitrary space-time dimensions.\nBy using the inversion formula of Caron-Huot and the integral (Mellin)\nrepresentation of conformal blocks, we show that the contribution from\nindividual exchanges to anomalous dimensions and corrections to the OPE\ncoefficients for "double-twist" operators\n$[\\mathcal{O}_1\\mathcal{O}_2]_{\\Delta,J}$ in $s-$channel can be written at\nfinite conformal spin in terms of generalized Wilson polynomials. This approach\nis democratic {\\it wrt} space-time dimensions, thus generalizing the earlier\nfindings to cases where closed form expressions of the conformal blocks are not\navailable.\n']. 
 Authors : ['Carlos Cardona, Sunny Guha, Surya Kir


 17) With TFIDF score 0.11292888224124908 and GPT-2 rank 76 
 Paper title : ['Regge meets collinear in strongly-coupled $\\mathcal{N} = 4$ super\n  Yang-Mills']. 
 Abstract : ['  We revisit the calculation of the six-gluon remainder function in planar\n$\\mathcal{N} = 4$ super Yang-Mills theory from the strong coupling TBA in the\nmulti-Regge limit and identify an infinite set of kinematically subleading\nterms. These new terms can be compared to the strong coupling limit of the\nfinite-coupling expressions for the impact factor and the BFKL eigenvalue\nproposed by Basso et al. in arXiv:1407.3766, which were obtained from an\nanalytic continuation of the Wilson loop OPE. After comparing the results order\nby order in those subleading terms, we show that it is possible to precisely\nmap both formalisms onto each other. A similar calculation can be carried out\nfor the seven-gluon amplitude, the result of which shows that the central\nemission vertex does not become trivial at strong co


 25) With TFIDF score 0.11811813712120056 and GPT-2 rank 145 
 Paper title : ['Modular Bootstrap Revisited']. 
 Abstract : ['  We constrain the spectrum of two-dimensional unitary, compact conformal field\ntheories with central charge c > 1 using modular bootstrap. Upper bounds on the\ngap in the dimension of primary operators of any spin, as well as in the\ndimension of scalar primaries, are computed numerically as functions of the\ncentral charge using semi-definite programming. Our bounds refine those of\nHellerman and Friedan-Keller, and are in some cases saturated by known CFTs. In\nparticular, we show that unitary CFTs with c < 8 must admit relevant\ndeformations, and that a nontrivial bound on the gap of scalar primaries exists\nfor c < 25. We also study bounds on the dimension gap in the presence of twist\ngaps, bounds on the degeneracy of operators, and demonstrate how "extremal\nspectra" which maximize the degeneracy at the gap can be determined\nnumerically.\n']. 
 Authors 

In [32]:
fast_gpt_tfidf_recommender(sentence_model_second,"1212.3616")


 1) With TFIDF score 0.13549210131168365 and GPT-2 rank 2 
 Paper title : ['Inflection Point Quintessence']. 
 Abstract : ['  We examine models in which the accelerated expansion of the universe is\ndriven by a scalar field rolling near an inflection point in the potential. For\nthe simplest such models, in which the potential is of the form V(\\phi) = V_0 +\nV_3 (\\phi-\\phi_0)^3, the scalar field can either evolve toward \\phi = \\phi_0 at\nlate times, yielding an asymptotic de Sitter expansion, or it can transition\nthrough the inflection point, producing a transient period of acceleration. We\ndetermine the parameter ranges which produce each of these two possibilities\nand also map out the region in parameter space for which the equation of state\nof the scalar field is close to -1 at all times up to the present, mimicking\n\\LambdaCDM. We show that the latter can be consistent with either eternal or\ntransient acceleration. More complicated inflection point models are also\ninve


 9) With TFIDF score 0.11292888224124908 and GPT-2 rank 56 
 Paper title : ['Regge meets collinear in strongly-coupled $\\mathcal{N} = 4$ super\n  Yang-Mills']. 
 Abstract : ['  We revisit the calculation of the six-gluon remainder function in planar\n$\\mathcal{N} = 4$ super Yang-Mills theory from the strong coupling TBA in the\nmulti-Regge limit and identify an infinite set of kinematically subleading\nterms. These new terms can be compared to the strong coupling limit of the\nfinite-coupling expressions for the impact factor and the BFKL eigenvalue\nproposed by Basso et al. in arXiv:1407.3766, which were obtained from an\nanalytic continuation of the Wilson loop OPE. After comparing the results order\nby order in those subleading terms, we show that it is possible to precisely\nmap both formalisms onto each other. A similar calculation can be carried out\nfor the seven-gluon amplitude, the result of which shows that the central\nemission vertex does not become trivial at strong cou


 17) With TFIDF score 0.21330490708351135 and GPT-2 rank 169 
 Paper title : ['Anomalous dimensions at finite conformal spin from OPE inversion']. 
 Abstract : ['  We compute anomalous dimensions of higher spin operators in Conformal Field\nTheory at arbitrary space-time dimension by using the OPE inversion formula of\n\\cite{Caron-Huot:2017vep}, both from the position space representation as well\nas from the integral (viz. Mellin) representation of the conformal blocks. The\nMellin space is advantageous over the position space not only in allowing to\nwrite expressions agnostic to the space-time dimension, but also in that it\nreplaces tedious recursion relations in terms of simple sums which are easy to\nperform. We evaluate the contributions of scalar and spin exchanges in the\n$t-$channel exactly, in terms of higher order Hypergeometric functions. These\nrelate to a particular exchange of conformal spin $\\beta=\\Delta+J$ in the\n$s-$channel through the inversion formula. Our exa


 25) With TFIDF score 0.14920838177204132 and GPT-2 rank 253 
 Paper title : ['The Multi-Regge Limit from the Wilson Loop OPE']. 
 Abstract : ['  The finite remainder function for planar, color-ordered, maximally helicity\nviolating scattering processes in N=4 super Yang-Mills theory possesses a\nnon-vanishing multi-Regge limit that depends on the choice of a Mandelstam\nregion. We analyze the combined multi-Regge collinear limit in all Mandelstam\nregions through an analytic continuation of the Wilson loop OPE. At leading\norder, the former is determined by the gluon excitation of the\nGubser-Klebanov-Polyakov string. We illustrate the general procedure at the\nexample of the heptagon remainder function at two loops. In this case, the\ncontinuation of the leading order terms in the Wilson loop OPE suffices to\ndetermine the two-loop multi-Regge heptagon functions in all Mandelstam regions\nfrom their symbols. The expressions we obtain are fully consistent with recent\nresults by Del 

In [39]:
fast_roberta_recommender(sentence_model_average,"1212.3616",10,0.2)


 1) With TFIDF score 0.4085037410259247 and Roberta rank 26 
 Paper title : ['Towards a Bootstrap approach to higher orders of epsilon expansion']. 
 Abstract : ['  We employ a hybrid approach in determining the anomalous dimension and OPE\ncoefficient of higher spin operators in the Wilson-Fisher theory. First we do a\nlarge spin analysis for CFT data where we use results obtained from the usual\nand the Mellin Bootstrap and also from Feynman diagram literature. This gives\nnew predictions at $O(\\epsilon^4)$ and $O(\\epsilon^5)$ for anomalous dimensions\nand OPE coefficients, and also provides a cross-check for the results from\nMellin Bootstrap. These higher orders get contributions from all higher spin\noperators in the crossed channel. We also use the Bootstrap in Mellin space\nmethod for $\\phi^3$ in $d=6-\\epsilon$ CFT where we calculate general higher\nspin OPE data. We demonstrate a higher loop order calculation in this approach\nby summing over contributions from higher spin

KeyboardInterrupt: 

In [12]:
recommendgpt(sentence_model_e2,"1908.10996",30)

/home/ubuntu/anaconda3/envs/aws_neuron_tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


1) Paper title : ['Geometric secret sharing in a model of Hawking radiation']. 
 Abstract : ['  We consider a black hole in three dimensional AdS space entangled with an\nauxiliary radiation system. We model the microstates of the black hole in terms\nof a field theory living on an end of the world brane behind the horizon, and\nallow this field theory to itself have a holographic dual geometry. This\ngeometry is also a black hole since entanglement of the microstates with the\nradiation leaves them in a mixed state. This "inception black hole" can be\npurified by entanglement through a wormhole with an auxiliary system which is\nnaturally identified with the external radiation, giving a realization of the\nER=EPR scenario. In this context, we propose an extension of the Ryu-Takayanagi\n(RT) formula, in which extremal surfaces computing entanglement entropy are\nallowed to pass through the brane into its dual geometry. This new rule\nreproduces the Page curve for evaporating black hole

7) Paper title : ['Effective entropy of quantum fields coupled with gravity']. 
 Abstract : ['  Entanglement entropy quantifies the amount of uncertainty of a quantum state.\nFor quantum fields in curved space, entanglement entropy of the quantum field\ntheory degrees of freedom is well-defined for a fixed background geometry. In\nthis paper, we propose a generalization of the quantum field theory\nentanglement entropy by including dynamical gravity. The generalized quantity\nnamed effective entropy, and its Renyi entropy generalizations, are defined by\nanalytic continuation of a gravitational path integral on replica geometry with\na co-dimension-$2$ brane at the boundary of region we are studying. We discuss\ndifferent approaches to define the region in a gauge invariant way, and show\nthat the effective entropy satisfies the quantum extremal surface formula. When\nthe quantum fields carry a significant amount of entanglement, the quantum\nextremal surface can have a topology transi

12) Paper title : ['Do We Understand Black Hole Entropy ?']. 
 Abstract : ["  I review various proposals for the nature of black hole entropy and for the\nmechanism behind the operation of the generalized second law. I stress the\nmerits of entanglement entropy {\\tenit qua\\/} black hole entropy, and point out\nthat, from an operational viewpoint, entanglement entropy is perfectly finite.\nProblems with this identification such as the multispecies problem and the\ntrivialization of the information puzzle are mentioned. This last leads me to\nassociate black hole entropy rather with the multiplicity of density operators\nwhich describe a black hole according to exterior observers. I relate this\nidentification to Sorkin's proof of the generalized second law. I discuss in\nsome depth Frolov and Page's proof of the same law, finding it relevant only\nfor scattering of microsystems by a black hole. Assuming that the law is\ngenerally valid I make evident the existence of the universal bou

18) Paper title : ['Holographic entanglement entropy of semi-local quantum liquids']. 
 Abstract : ['  We consider the holographic entanglement entropy of $(d+2)$-dimensional\nsemi-local quantum liquids, for which the dual gravity background in the deep\ninterior is $AdS_{2}\\times\\mathbb{R}^{d}$ multiplied by a warp factor which\ndepends on the radial coordinate. The entropy density of this geometry goes to\nzero in the extremal limit. The thermodynamics associated with this semi-local\nbackground is discussed via dimensional analysis and scaling arguments. For the\ncase of an asymptotically AdS UV completion of this geometry, we show that the\nentanglement entropy of a strip and an annulus exhibits a phase transition as a\ntypical length of the different shapes is varied, while there is no sign of\nsuch a transition for the entanglement entropy of a sphere. Moreover, for the\nspherical entangling region, the leading order contribution to the entanglement\nentropy in the IR is calcul

24) Paper title : ['Renormalization group flow of entanglement entropy on spheres']. 
 Abstract : ["  We explore entanglement entropy of a cap-like region for a generic quantum\nfield theory residing in the Bunch-Davies vacuum on de Sitter space.\nEntanglement entropy in our setup is identical with the thermal entropy in the\nstatic patch of de Sitter, and we derive a simple relation between the vacuum\nexpectation value of the energy-momentum tensor trace and the RG flow of\nentanglement entropy. In particular, renormalization of the cosmological\nconstant and logarithmic divergence of the entanglement entropy are\ninterrelated in our setup. We confirm our findings by recovering known\nuniversal contributions for a free field theory deformed by a mass operator as\nwell as obtain correct universal behaviour at the fixed points. Simple examples\nof entanglement entropy flows are elaborated in $d=2,3,4$. In three dimensions\nwe find that while the renormalized entanglement entropy is sta

30) Paper title : ['Evaporating Black Holes Coupled to a Thermal Bath']. 
 Abstract : ['  We study the doubly holographic model of [arXiv:1908.10996] in the situation\nwhere a black hole in two-dimensional JT gravity theory is coupled to an\nauxiliary bath system at arbitrary finite temperature. Depending on the initial\ntemperature of the black hole relative to the bath temperature, the black hole\ncan lose mass by emitting Hawking radiation, stay in equilibrium with the bath\nor gain mass by absorbing thermal radiation from the bath. In all of these\nscenarios, a unitary Page curve is obtained by applying the usual prescription\nfor holographic entanglement entropy and identifying the quantum extremal\nsurface for the generalized entropy, using both analytical and numeric\ncalculations. As the application of the entanglement wedge reconstruction, we\nfurther investigate the reconstruction of the black hole interior from a\nsubsystem containing the Hawking radiation. We examine the ro

In [32]:
fastlstmrecommender(sentence_model_e2,"1908.10996")


 1). With TFIDF rank 0.563462495803833 and LSTM rank 0 
 ABSTRACT === ['  We consider a black hole in three dimensional AdS space entangled with an\nauxiliary radiation system. We model the microstates of the black hole in terms\nof a field theory living on an end of the world brane behind the horizon, and\nallow this field theory to itself have a holographic dual geometry. This\ngeometry is also a black hole since entanglement of the microstates with the\nradiation leaves them in a mixed state. This "inception black hole" can be\npurified by entanglement through a wormhole with an auxiliary system which is\nnaturally identified with the external radiation, giving a realization of the\nER=EPR scenario. In this context, we propose an extension of the Ryu-Takayanagi\n(RT) formula, in which extremal surfaces computing entanglement entropy are\nallowed to pass through the brane into its dual geometry. This new rule\nreproduces the Page curve for evaporating black holes, consistently with 


 7). With TFIDF rank 0.40121328830718994 and LSTM rank 6 
 ABSTRACT === ['  Entanglement entropy quantifies the amount of uncertainty of a quantum state.\nFor quantum fields in curved space, entanglement entropy of the quantum field\ntheory degrees of freedom is well-defined for a fixed background geometry. In\nthis paper, we propose a generalization of the quantum field theory\nentanglement entropy by including dynamical gravity. The generalized quantity\nnamed effective entropy, and its Renyi entropy generalizations, are defined by\nanalytic continuation of a gravitational path integral on replica geometry with\na co-dimension-$2$ brane at the boundary of region we are studying. We discuss\ndifferent approaches to define the region in a gauge invariant way, and show\nthat the effective entropy satisfies the quantum extremal surface formula. When\nthe quantum fields carry a significant amount of entanglement, the quantum\nextremal surface can have a topology transition, after which a


 12). With TFIDF rank 0.24224503338336945 and LSTM rank 11 
 ABSTRACT === ["  I review various proposals for the nature of black hole entropy and for the\nmechanism behind the operation of the generalized second law. I stress the\nmerits of entanglement entropy {\\tenit qua\\/} black hole entropy, and point out\nthat, from an operational viewpoint, entanglement entropy is perfectly finite.\nProblems with this identification such as the multispecies problem and the\ntrivialization of the information puzzle are mentioned. This last leads me to\nassociate black hole entropy rather with the multiplicity of density operators\nwhich describe a black hole according to exterior observers. I relate this\nidentification to Sorkin's proof of the generalized second law. I discuss in\nsome depth Frolov and Page's proof of the same law, finding it relevant only\nfor scattering of microsystems by a black hole. Assuming that the law is\ngenerally valid I make evident the existence of the universal bo


 18). With TFIDF rank 0.3378855586051941 and LSTM rank 17 
 ABSTRACT === ['  We consider the holographic entanglement entropy of $(d+2)$-dimensional\nsemi-local quantum liquids, for which the dual gravity background in the deep\ninterior is $AdS_{2}\\times\\mathbb{R}^{d}$ multiplied by a warp factor which\ndepends on the radial coordinate. The entropy density of this geometry goes to\nzero in the extremal limit. The thermodynamics associated with this semi-local\nbackground is discussed via dimensional analysis and scaling arguments. For the\ncase of an asymptotically AdS UV completion of this geometry, we show that the\nentanglement entropy of a strip and an annulus exhibits a phase transition as a\ntypical length of the different shapes is varied, while there is no sign of\nsuch a transition for the entanglement entropy of a sphere. Moreover, for the\nspherical entangling region, the leading order contribution to the entanglement\nentropy in the IR is calculated analytically. It exh


 24). With TFIDF rank 0.3242841064929962 and LSTM rank 23 
 ABSTRACT === ["  We explore entanglement entropy of a cap-like region for a generic quantum\nfield theory residing in the Bunch-Davies vacuum on de Sitter space.\nEntanglement entropy in our setup is identical with the thermal entropy in the\nstatic patch of de Sitter, and we derive a simple relation between the vacuum\nexpectation value of the energy-momentum tensor trace and the RG flow of\nentanglement entropy. In particular, renormalization of the cosmological\nconstant and logarithmic divergence of the entanglement entropy are\ninterrelated in our setup. We confirm our findings by recovering known\nuniversal contributions for a free field theory deformed by a mass operator as\nwell as obtain correct universal behaviour at the fixed points. Simple examples\nof entanglement entropy flows are elaborated in $d=2,3,4$. In three dimensions\nwe find that while the renormalized entanglement entropy is stationary at the\nfixed po


 30). With TFIDF rank 0.354154109954834 and LSTM rank 29 
 ABSTRACT === ['  We study the doubly holographic model of [arXiv:1908.10996] in the situation\nwhere a black hole in two-dimensional JT gravity theory is coupled to an\nauxiliary bath system at arbitrary finite temperature. Depending on the initial\ntemperature of the black hole relative to the bath temperature, the black hole\ncan lose mass by emitting Hawking radiation, stay in equilibrium with the bath\nor gain mass by absorbing thermal radiation from the bath. In all of these\nscenarios, a unitary Page curve is obtained by applying the usual prescription\nfor holographic entanglement entropy and identifying the quantum extremal\nsurface for the generalized entropy, using both analytical and numeric\ncalculations. As the application of the entanglement wedge reconstruction, we\nfurther investigate the reconstruction of the black hole interior from a\nsubsystem containing the Hawking radiation. We examine the roles of the Ha


 36). With TFIDF rank 0.29959219694137573 and LSTM rank 35 
 ABSTRACT === ['  We examine the idea that in quantum gravity, the entanglement entropy of a\ngeneral region should be finite and the leading contribution is given by the\nBekenstein-Hawking area law. Using holographic entanglement entropy\ncalculations, we show that this idea is realized in the Randall-Sundrum II\nbraneworld for sufficiently large regions in smoothly curved backgrounds.\nExtending the induced gravity action on the brane to include the\ncurvature-squared interactions, we show that the Wald entropy closely matches\nthe expression describing the entanglement entropy. The difference is that for\na general region, the latter includes terms involving the extrinsic curvature\nof the entangling surface, which do not appear in the Wald entropy. We also\nconsider various limitations on the validity of these results.\n'] 
 TITLE ==== ['On Spacetime Entanglement'] 
 ARXIV_ID ==== ['1304.2030'] 
 


 36

 37). With TFIDF


 42). With TFIDF rank 0.333359032869339 and LSTM rank 41 
 ABSTRACT === ['  We compute the time-dependent entanglement entropy of a CFT which starts in\nrelatively simple initial states. The initial states are the thermofield double\nfor thermal states, dual to eternal black holes, and a particular pure state,\ndual to a black hole formed by gravitational collapse. The entanglement entropy\ngrows linearly in time. This linear growth is directly related to the growth of\nthe black hole interior measured along "nice" spatial slices. These nice slices\nprobe the spacelike direction in the interior, at a fixed special value of the\ninterior time. In the case of a two-dimensional CFT, we match the bulk and\nboundary computations of the entanglement entropy. We briefly discuss the long\ntime behavior of various correlators, computed via classical geodesics or\nsurfaces, and point out that their exponential decay comes about for similar\nreasons. We also present the time evolution of the wav

KeyboardInterrupt: 

In [60]:
recommend("1908.00015",25) #pure tf-idf results

 
 1) A note on defect Mellin amplitudes by Vasco Goncalves, Georgios Itsios (0.65971667). Arxiv: 1803.06721
 
 2) The Next $16$ Higher Spin Currents and Three-Point Functions in the
  Large ${\cal N}=4$ Holography by Changhyun Ahn, Dong-gyu Kim and Man Hea Kim (0.6346596). Arxiv: 1703.01744
 
 3) Bootstrapping string theory on AdS$_5 \times S^5$ by J. M. Drummond, H. Paul and M. Santagata (0.63369775). Arxiv: 2004.07282
 
 4) Nonperturbative Mellin Amplitudes: Existence, Properties, Applications by Joao Penedones, Joao A. Silva, Alexander Zhiboedov (0.6324333). Arxiv: 1912.11100
 
 5) OPE inversion in Mellin space by Carlos Cardona (0.6264813). Arxiv: 1803.05086
 
 6) The Operator Product Expansions in the ${\cal N}=4$ Orthogonal Wolf
  Space Coset Model by Changhyun Ahn, Man Hea Kim and Jinsub Paeng (0.6253749). Arxiv: 1904.06855
 
 7) Towards Spinning Mellin Amplitudes by Heng-Yu Chen, En-Jui Kuo and Hideki Kyono (0.6166204). Arxiv: 1712.07991
 
 8) Towards a Bootstrap approach to h

In [58]:
fastlstmrecommender(sentence_model_e2,"1908.00015")


 1). With TFIDF rank 0.17074674367904663 and LSTM rank 1 
2) Paper title : ['Quantum vortices, M2-branes and black holes']. 
 Abstract : ['  We study the partition functions of BPS vortices and magnetic monopole\noperators, in gauge theories describing $N$ M2-branes. In particular, we\nexplore two closely related methods to study the Cardy limit of the index on\n$S^2\\times\\mathbb{R}$. The first method uses the factorization of this index to\nvortex partition functions, while the second one uses a continuum approximation\nfor the monopole charge sums. Monopole condensation confines most of the $N^2$\ndegrees of freedom except $N^{\\frac{3}{2}}$ of them, even in the high\ntemperature deconfined phase. The resulting large $N$ free energy statistically\naccounts for the Bekenstein-Hawking entropy of large BPS black holes in\n$AdS_4\\times S^7$. Our Cardy free energy also suggests a finite $N$ version of\nthe $N^{\\frac{3}{2}}$ degrees of freedom.\n']. 
 Authors : ['Sunjin Choi, Chiung H


 8). With TFIDF rank 0.1806628555059433 and LSTM rank 39 
9) Paper title : ['Conformally Soft Photons and Gravitons']. 
 Abstract : ['  The four-dimensional $S$-matrix is reconsidered as a correlator on the\ncelestial sphere at null infinity. Asymptotic particle states can be\ncharacterized by the point at which they enter or exit the celestial sphere as\nwell as their $SL(2,\\mathbb C)$ Lorentz quantum numbers: namely their conformal\nscaling dimension and spin $h\\pm \\bar h$ instead of the energy and momentum.\nThis characterization precludes the notion of a soft particle whose energy is\ntaken to zero. We propose it should be replaced by the notion of a conformally\nsoft particle with $h=0$ or $\\bar h=0$. For photons we explicitly construct\nconformally soft $SL(2,\\mathbb C)$ currents with dimensions $(1,0)$ and\nidentify them with the generator of a $U(1)$ Kac-Moody symmetry on the\ncelestial sphere. For gravity the generator of celestial conformal symmetry is\nconstructed from


 13). With TFIDF rank 0.1734973043203354 and LSTM rank 65 
14) Paper title : ['Quantum Information Metric on $\\mathbb{R} \\times S^{d-1}$']. 
 Abstract : ['  We present a formula for the information metric on $\\mathbb{R} \\times\n{S}^{d-1}$ for a scalar primary operator of integral dimension $\\Delta \\, (\\,\\,\n> \\frac{d+1}{2})$. This formula is checked for various space-time dimensions\n$d$ and $\\Delta$ in the field theory side. We check the formula in the gravity\nside using the holographic setup. We clarify the regularization and\nrenormalization involved in these computations. We also show that the quantum\ninformation metric of an exactly marginal operator agrees with the leading\norder of the interface free energy of the conformal Janus on Euclidean ${S}^d$,\nwhich is checked for $d=2, 3$.\n']. 
 Authors : ['Dongsu Bak and Andrea Trivella']. 
 Arxiv ID : ['1707.05366'] 
 
 

 13

 14). With TFIDF rank 0.2037607729434967 and LSTM rank 70 
15) Paper title : ['Holographic Rec


 19). With TFIDF rank 0.3393406271934509 and LSTM rank 113 
20) Paper title : ['Polyakov-Mellin Bootstrap for AdS loops']. 
 Abstract : ['  We consider holographic CFTs and study their large $N$ expansion. We use\nPolyakov-Mellin bootstrap to extract the CFT data of all operators, including\nscalars, till $O(1/N^4)$. We add a contact term in Mellin space, which\ncorresponds to an effective $\\phi^4$ theory in AdS and leads to anomalous\ndimensions for scalars at $O(1/N^2)$. Using this we fix $O(1/N^4)$ anomalous\ndimensions for double trace operators finding perfect agreement with\n\\cite{loopal} (for $\\Delta_{\\phi}=2$). Our approach generalizes this to any\ndimensions and any value of conformal dimensions of external scalar field. In\nthe second part of the paper, we compute the loop amplitude in AdS which\ncorresponds to non-planar correlators of in CFT. More precisely, using CFT data\nat $O(1/N^4)$ we fix the AdS bubble diagram and the triangle diagram for the\ngeneral case.\n'].


 25). With TFIDF rank 0.16374078392982483 and LSTM rank 187 
26) Paper title : ['A New Constraint on Effective Field Theories of the QCD Flux Tube']. 
 Abstract : ['  Effective magnetic $SU(N)$ gauge theory with classical $Z_N$ flux tubes of\nintrinsic width $\\frac{1}{M}$ is an effective field theory of the long distance\nquark-antiquark interaction in $SU(N)$ Yang-Mills theory. Long wavelength\nfluctuations of the $Z_N$ vortices of this theory lead to an effective string\ntheory. In this paper we clarify the connection between effective field theory\nand effective string theory and we propose a new constraint on these vortices.\nWe first examine the impact of string fluctuations on the classical dual\nsuperconductor description of confinement. At inter-quark distances $R\\sim\n\\frac{1}{M}$ the classical action for a straight flux tube determines the heavy\nquark potentials. At distances $R \\gg \\frac{1}{M}$ fluctuations of the flux\ntube axis $\\tilde{x}$ give rise to an effective

In [31]:
recommendgpt("1908.00015",30)

/home/ubuntu/anaconda3/envs/aws_neuron_tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


0) Paper title : ['Modular Invariance in Superstring Theory From ${\\cal N} = 4$\n  Super-Yang-Mills']. 
 Abstract : ['  We study the four-point function of the lowest-lying half-BPS operators in\nthe ${\\cal N} =4$ $SU(N)$ super-Yang-Mills theory and its relation to the\nflat-space four-graviton amplitude in type IIB superstring theory. We work in a\nlarge-$N$ expansion in which the complexified Yang-Mills coupling $\\tau$ is\nfixed. In this expansion, non-perturbative instanton contributions are present,\nand the $SL(2, \\mathbb{Z})$ duality invariance of correlation functions is\nmanifest. Our results are based on a detailed analysis of the sphere partition\nfunction of the mass-deformed SYM theory, which was previously computed using\nsupersymmetric localization. This partition function determines a certain\nintegrated correlator in the undeformed ${\\cal N} = 4$ SYM theory, which in\nturn constrains the four-point correlator at separated points. In a\nnormalization where the two-p

6) Paper title : ['Platonic Field Theories']. 
 Abstract : ['  We study renormalization group (RG) fixed points of scalar field theories\nendowed with the discrete symmetry groups of regular polytopes. We employ the\nfunctional perturbative renormalization group (FPRG) approach and the\n$\\epsilon$-expansion in $d=d_c-\\epsilon$. The upper critical dimensions\nrelevant to our analysis are $d_c =\n6,4,\\frac{10}{3},3,\\frac{14}{5},\\frac{8}{3},\\frac{5}{2},\\frac{12}{5}$; in order\nto get access to the corresponding RG beta functions, we derive general\nmulticomponent beta functionals $\\beta_V$ and $\\beta_Z$ in the aforementioned\nupper critical dimensions, most of which are novel. The field theories we\nanalyze have $N=2$ (polygons), $N=3$ (Platonic solids) and $N=4$\n(hyper-Platonic solids) field components. The main results of this analysis\ninclude a new candidate universality class in three physical dimensions based\non the symmetry group $\\mathbb{D}_5$ of the Pentagon. Moreover

12) Paper title : ['Exact $\\nabla^4 R^4$ couplings and helicity supertraces']. 
 Abstract : ['  In type II string theory compactified on a $d$-dimensional torus $T^d$ down\nto $D=10-d$ dimensions, the $R^4$ and $\\nabla^4 R^4$ four-graviton couplings\nare known exactly, for all values of the moduli, in terms of certain Eisenstein\nseries of the U-duality group $E_{d}(\\mathbb{Z})$. In the limit where one\ncircle in the torus becomes large, these couplings are expected to reduce to\ntheir counterpart in dimension $D+1$, plus threshold effects and exponentially\nsuppressed corrections corresponding to BPS black holes in dimension $D+1$\nwhose worldline winds around the circle. By combining the weak coupling and\nlarge radius limits, we determine these exponentially suppressed corrections\nexactly, and demonstrate that the contributions of 1/4-BPS black holes to the\n$\\nabla^4 R^4$ coupling are proportional to the appropriate helicity\nsupertrace. Mathematically, our results provide the

20) Paper title : ['Confinement and Pseudoscalar Glueball Spectrum in the $2+1$D QCD-Like\n  Theory from the Non-Susy D$2$ Brane']. 
 Abstract : ['  Here we study two important properties of $2+1$ dimensional QCD --\nconfinement and pseudoscalar glueball spectrum -- with holographic approach. We\nconsider the low energy decoupled geometry of the isotropic non-susy D$2$\nbrane. We find the corresponding gauge theory is similar to the $2+1$-dim\nYang-Mills theory with the running coupling $\\lambda^2$. At the extremal limit\n(i.e. BPS limit), this gauge theory reduces to the super-YM theory. From the\nNambu-Goto action of a test string, the potential of a \\qq pair located on the\nboundary is calculated. At large \\qq separation it gives the tension $\\sigma$\nof the QCD flux-tube. The parametric dependencies of $\\sigma$ is shown\npictorially. It is found that $\\sigma$ is a monotonically increasing function\nof the effective coupling $\\lambda^2$. In comparison,\n$\\sqrt{\\sigma}/g_\\t

26) Paper title : ['Induced QCD II: Numerical results']. 
 Abstract : ['  We numerically explore an alternative discretization of continuum\n$\\text{SU}(N_c)$ Yang-Mills theory on a Euclidean spacetime lattice, originally\nintroduced by Budzcies and Zirnbauer for gauge group $\\text{U}(N_c)$. This\ndiscretization can be reformulated such that the self-interactions of the gauge\nfield are induced by a path integral over $N_b$ auxiliary bosonic fields, which\ncouple linearly to the gauge field. In the first paper of the series we have\nshown that the theory reproduces continuum $\\text{SU}(N_c)$ Yang-Mills theory\nin $d=2$ dimensions if $N_b$ is larger than $N_c-\\frac{3}{4}$ and conjectured,\nfollowing the argument of Budzcies and Zirnbauer, that this remains true for\n$d>2$. In the present paper, we test this conjecture by performing lattice\nsimulations of the simplest nontrivial case, i.e., gauge group $\\text{SU}(2)$\nin three dimensions. We show that observables computed in the ind

In [62]:
recommend("1212.3616",30) #pure tf-idf results

 
 1) Implications of the AdS/CFT Correspondence on Spacetime and Worldsheet
  OPE Coefficients by Sudip Ghosh, Sourav Sarkar and Mritunjay Verma (0.74077374). Arxiv: 1703.06132
 
 2) Recursive construction of the operator product expansion in curved space by Markus B Fr\"ob (0.725186). Arxiv: 2007.15668
 
 3) The Operator Product Expansion between the 16 Lowest Higher Spin
  Currents in the N=4 Superspace by Changhyun Ahn and Man Hea Kim (0.70562214). Arxiv: 1509.01908
 
 4) Holographic OPE Coefficients from AdS Black Holes with Matters by Yue-Zhou Li, Zhan-Feng Mai, H. Lu (0.7030324). Arxiv: 1905.09302
 
 5) Solving M-theory with the Conformal Bootstrap by Nathan B. Agmon, Shai M. Chester, Silviu S. Pufu (0.6985825). Arxiv: 1711.07343
 
 6) Analytic bootstrap at large spin by Apratim Kaviraj, Kallol Sen and Aninda Sinha (0.69455445). Arxiv: 1502.01437
 
 7) Random Statistics of OPE Coefficients and Euclidean Wormholes by Alexandre Belin, Jan de Boer (0.6894057). Arxiv: 2006.05499
 
 

In [ ]:
#recommendgpt("1212.3616",30)

In [ ]:
#recommendgpt("2010.03560",30)

In [ ]:
#recommendgpt("1803.04434",30)

In [ ]:
#recommendgpt("1610.06186",30)

In [ ]:
#recommendgpt("1405.5137",30)

In [ ]:
#recommendgpt("2009.01246",30)

In [ ]:
#recommend("1703.00278",25)

In [ ]:
#recommendgpt(sentence_model_e2,"1703.00278",25)

In [70]:
fast_gpt_tfidf_recommender(sentence_model_e2,"1703.00278")


 1). With TFIDF rank 0.10662900656461716 and GPT-2 rank 2 
1) Paper title : ['Polyakov formulas for GJMS operators from AdS/CFT']. 
 Abstract : ['  We argue that the AdS/CFT calculational prescription for double-trace\ndeformations leads to a holographic derivation of the conformal anomaly, and\nits conformal primitive, associated to the whole family of conformally\ncovariant powers of the Laplacian (GJMS operators) at the conformal boundary.\nThe bulk side involves a quantum 1-loop correction to the SUGRA action and the\nboundary counterpart accounts for a sub-leading term in the large-N limit. The\nsequence of GJMS conformal Laplacians shows up in the two-point function of the\nCFT operator dual to a bulk scalar field at certain values of its scaling\ndimension. The restriction to conformally flat boundary metrics reduces the\nbulk computation to that of volume renormalization which renders the universal\ntype A anomaly. In this way, we directly connect two chief roles of the\nQ-cur


 7). With TFIDF rank 0.11777441203594208 and GPT-2 rank 25 
7) Paper title : ['Real-time finite-temperature correlators from AdS/CFT']. 
 Abstract : ["  In this paper we use AdS/CFT ideas in conjunction with insights from finite\ntemperature real-time field theory formalism to compute 3-point correlators of\n${\\cal N}{=}4$ super Yang-Mills operators, in real time and at finite\ntemperature. To this end, we propose that the gravity field action is\nintegrated only over the right and left quadrants of the Penrose diagram of the\nAnti de Sitter-Schwarzschild background, with a relative sign between the two\nterms. For concreteness we consider the case of a scalar field in the black\nhole background. Using the scalar field Schwinger-Keldysh bulk-to-boundary\npropagators, we give the general expression of a 3-point real-time Green's\ncorrelator. We then note that this particular prescription amounts to adapting\nthe finite-temperature analog of Veltman's circling rules to tree-level Witte


 12). With TFIDF rank 0.11869436502456665 and GPT-2 rank 40 
12) Paper title : ['The Ryu-Takayanagi Formula from Quantum Error Correction']. 
 Abstract : ['  I argue that a version of the quantum-corrected Ryu-Takayanagi formula holds\nin any quantum error-correcting code. I present this result as a series of\ntheorems of increasing generality, with the final statement expressed in the\nlanguage of operator-algebra quantum error correction. In AdS/CFT this gives a\n"purely boundary" interpretation of the formula. I also extend a recent\ntheorem, which established entanglement-wedge reconstruction in AdS/CFT, when\ninterpreted as a subsystem code, to the more general, and I argue more\nphysical, case of subalgebra codes. For completeness, I include a\nself-contained presentation of the theory of von Neumann algebras on\nfinite-dimensional Hilbert spaces, as well as the algebraic definition of\nentropy. The results confirm a close relationship between bulk gauge\ntransformations, edge-m


 17). With TFIDF rank 0.12236008793115616 and GPT-2 rank 54 
17) Paper title : ['Numerical study of the Lorentzian Engle-Pereira-Rovelli-Livine spin foam\n  amplitude']. 
 Abstract : ['  The Lorentzian EPRL spin foam amplitude for loop quantum gravity is a\nmulti-dimensional non-compact integral of highly oscillating functions. Using a\nmethod based on the decomposition of Clebsch-Gordan coefficients for the\nunitary infinite-dimensional representations of SL(2,C) in terms of those of\nSU(2), we are able to provide for the first time numerical evaluations of the\nvertex amplitude. The values obtained support the asymptotic formula obtained\nby Barrett and collaborators with a saddle point approximation, showing, in\nparticular, a power-law decay and oscillations related to the Regge action. The\ncomparison offers a test of the efficiency of the method. Truncating the\ndecomposition to the first few terms provides a qualitative matching of the\npower-law decay and oscillations. For vec


 22). With TFIDF rank 0.10977703332901001 and GPT-2 rank 87 
22) Paper title : ['Double Trace Flows and Holographic RG in dS/CFT correspondence']. 
 Abstract : ["  If there is a dS/CFT correspondence, time evolution in the bulk should\ntranslate to RG flows in the dual euclidean field theory. Consequently,\nalthough the dual field is expected to be non-unitary, its RG flows will carry\nan imprint of the unitary time evolution in the bulk. In this note we examine\nthe prediction of holographic RG in de Sitter space for the flow of double and\ntriple trace couplings in any proposed dual. We show quite generally that the\ncorrect form of the field theory beta functions for the double trace couplings\nis obtained from holography, provided one identifies the scale of the field\ntheory with (i|T|) where T is the `time' in conformal coordinates. For dS(4),\nwe find that with an appropriate choice of operator normalization, it is\npossible to have real n-point correlation functions as well as


 27). With TFIDF rank 0.11537638306617737 and GPT-2 rank 110 
27) Paper title : ['Subleading Regge limit from a soft anomalous dimension']. 
 Abstract : ['  Wilson lines capture important features of scattering amplitudes, for example\nsoft effects relevant for infrared divergences, and the Regge limit. Beyond the\nleading power approximation, corrections to the eikonal picture have to be\ntaken into account. In this paper, we study such corrections in a model of\nmassive scattering amplitudes in N = 4 super Yang-Mills, in the planar limit,\nwhere the mass is generated through a Higgs mechanism. Using known three-loop\nanalytic expressions for the scattering amplitude, we find that the first power\nsuppressed term has a very simple form, equal to a single power law. We propose\nthat its exponent is governed by the anomalous dimension of a Wilson loop with\na scalar inserted at the cusp, and we provide perturbative evidence for this\nproposal. We also analyze other limits of the amplit